# zad 4 Jakub Iliński

In [74]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Tuple, Dict
import torch
import torch.nn.functional as F 
from tqdm import tqdm

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

device(type='cuda', index=0)

In [75]:
queries: List[str] = []
with open('task4_questions.txt') as file:
    for q in file:
        queries.append(q)
queries.sort()
print(*queries)

Adam Cisowski to postać z której powieści Kornela Makuszyńskiego?
 Adwersarz to sprzymierzeniec czy przeciwnik?
 Afryka leży w większości na półkuli wschodniej czy zachodniej?
 Akcja której komedii rozgrywa się częściowo w Szuflandii?
 Akcyza to podatek pośredni czy bezpośredni?
 Amoniak jest cieczą czy gazem?
 Andantino to tempo szybsze czy wolniejsze od andante?
 Anion posiada przewagę elektronów czy protonów?
 Antoni Patek, pionier produkcji dokładnych zegarków w Szwajcarii, był uczestnikiem którego powstania?
 Aorta wychodzi z serca czy do niego wchodzi?
 Apokalipsa św. Jana to księga dydaktyczna, historyczna czy prorocza?
 Argentyna, Boliwia, Brazylia, Chile – w którym z tych krajów hiszpański nie jest językiem urzędowym?
 Armię którego państwa Napoleon rozbił pod Jeną w 1806 r.?
 Atos i Portos to imiona dwóch muszkieterów – a jak się nazywał trzeci?
 Auksyny to hormony wzrostowe roślin czy zwierząt?
 Automat do gry poruszany za pomocą znajdującej się z boku dźwigni to inaczej „je

### grupy:
1. czy: [tak, nie]
2. ile / Z ilu: [liczba]
3. _ : [normalny prompt]

In [76]:
def group_querry(q: str) -> int:
    if q.lower()[:3] == 'czy': return 1
    elif q.lower()[:3] == 'ile' or q.lower()[:5] == 'z ilu': return 2
    else: return 3
    
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def get_prob(context: str, label: str) -> float:
    full = context + label
    
    input_ids_full = tokenizer(full, return_tensors='pt')['input_ids'].to(DEVICE)
    input_ids_context = tokenizer(context, return_tensors='pt')['input_ids'].to(DEVICE)
    
    with torch.no_grad():
        output = model(input_ids=input_ids_full)
        logits, labels = output.logits[:, :-1, :], input_ids_full[:, 1:]
        log_probs = log_probs_from_logits(logits, labels)
        
    context_len, full_len = input_ids_context.size(1), input_ids_full.size(1)
    start, end = max(context_len - 1, 0), full_len - 1
    
    label_log_probs = log_probs[:, start:end]
    return float(label_log_probs.mean().item())

def generate_candidates(prompt: str, num: int, max_new_tokens: int) -> List[str]:
    input_ids = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
    out = model.generate(
        **input_ids,
        no_repeat_ngram_size=3,
        temperature=0.8,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        num_return_sequences=num,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    texts: List[str] = []
    for seq in out:
        full = tokenizer.decode(seq, skip_special_tokens=True)
        new = full[len(prompt):]
        texts.append(new.strip())
        
    return texts

In [77]:
def ans_querry_g1(query: str, mode: str) -> str:
    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Czy pingwin potrafi latać?", "Odpowieź: nie"),
        ],
        "FEW_SHOT": [
            ("Czy pingwin potrafi latać?", "Odpowiedź: nie"),
            ("Czy złoto jest metalem szlachetnym?", "Odpowiedź: tak"),
            ("Czy Sahara leży w Ameryce Południowej?", "Odpowiedź: nie"),
            ("Czy liczba 0 jest liczbą parzystą?", "Odpowiedź: tak"),
            ("Czy Mount Everest znajduje się w Europie?", "Odpowiedź: nie"),
            ("Czy woda wrze w temperaturze 100 stopni Celsjusza przy ciśnieniu normalnym?", "Odpowiedź: tak"),
            ("Czy człowiek ma trzy płuca?", "Odpowiedź: nie"),
            ("Czy Polska ma dostęp do morza?", "Odpowiedź: tak"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a}\n"
    
    prob_yes = get_prob(f"{context} {query} Odpowiedź: ", "tak")
    prob_no = get_prob(f"{context} {query} Odpowiedź: ", "nie")
    return "tak" if prob_yes > prob_no else "nie"

def ans_querry_g2(query: str, mode: str) -> str:
    def pick_best_candidate(candidates: List[str]) -> str:
        ans: List[Tuple[int, str]] = []
        for candidate in candidates:
            score = 0
            for char in candidate:
                if not ord('0') <= ord(char) <= ord('9'):
                    score -= 1
            ans.append((score, candidate))
            
        return sorted(ans, reverse=True)[0][1]

    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Ile dni ma rok przestępny?", "366"),
        ],
        "FEW_SHOT": [
            ("Ile dni ma rok przestępny?", "366"),
            ("Ile kontynentów wyróżnia się na Ziemi?", "7"),
            ("Ile graczy liczy drużyna siatkówki na boisku?", "6"),
            ("Ile boków ma sześciokąt foremny?", "6"),
            ("Ile kilometrów liczy maraton olimpijski?", "42.195"),
            ("Ile bierek ma komplet szachowy na początku partii?", "32"),
            ("Ile minut trwa trzy kwadranse?", "45"),
            ("Ile planet liczy Układ Słoneczny?", "8"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a}\n"
        
    candidates: List[str] = generate_candidates(f"{context}{query}", 3, 25)
    return pick_best_candidate(candidates)

def ans_querry_g3(query: str, mode: str) -> str:
    def pick_best_candidate(candidates: List[str]) -> str:
        ans: List[Tuple[int, str]] = []
        for candidate in candidates:
            score = 0
            for char in candidate:
                if not ord('a') <= ord(char) <= ord('z') and not char in ['ą', 'ę', 'ź', 'ć', 'ś']:
                    score -= 1
            ans.append((score, candidate))
            
        return max(ans)[1]
    
    shots: Dict[str, List[Tuple[str, str]]] = {
        "NO_SHOT": [], 
        "ONE_SHOT": [
            ("Jakie miasto leży najbliżej Morskiego Oka?", "Zakopane"),
        ],
        "FEW_SHOT": [
            ("Jakie miasto leży najbliżej Morskiego Oka?", "Zakopane"),
            ("Jak nazywa się najstarszy polski uniwersytet?", "Uniwersytet Jagielloński"),
            ("Jakie miasto leży nad rzeką Brdą?", "Bydgoszcz"),
            ("Które polskie miasto nazywane jest 'stolicą Tatr'?", "Zakopane"),
            ("Jak nazywa się park narodowy położony w Tatrach?", "Tatrzański Park Narodowy"),
            ("Jakie miasto nazywane jest 'Perłą Renesansu'?", "Zamość"),
            ("Jak nazywa się najdłuższy tunel w Polsce?", "Tunel Zakopianka"),
            ("Jak nazywa się najgłębsze jezioro w Polsce?", "Hańcza"),
        ],
    }
    
    context = ""
    for q, a in shots[mode]:
        context += f"{q} {a}\n"
        
    candidates: List[str] = generate_candidates(f"{context}{query}", 7, 25)
    return pick_best_candidate(candidates)

In [78]:
def ans_querry(q: str, mode) -> str:
    match group_querry(q):
        case 1: return ans_querry_g1(q, mode)
        case 2: return ans_querry_g2(q, mode)
        case _: return ans_querry_g3(q, mode)

MODE = "ONE_SHOT" # "NO_SHOT", "ONE_SHOT", "FEW_SHOT"
with open('task4_questions.txt', 'r') as file, open('found_answers.txt', 'w') as ansf:
    for line in tqdm(file):
        ans = ans_querry(line, MODE)
        ansf.write(f"{ans}\n")
        # print(f"{line}{ans}\n\n")

2500it [10:15,  4.06it/s]


In [79]:
import editdistance
import sys


def scaled_editdist(ans, cor):
    ans = ans.lower()
    cor = cor.lower()
    
    return editdistance.eval(ans, cor) / len(cor)
    
def single_match(a, c):
    if c.isdecimal():
        return a == c
    return scaled_editdist(a, c) < 0.5
        
def match(ans, cor):
    return any(single_match(ans, c) for c in cor)
        
found_answers = []
correct_answers = []

for x in open('task4_answers.txt'):
    x = x.strip()
    correct_answers.append(x.lower().split('\t'))
    
for x in open('found_answers.txt'):    
    x = x.strip()
    found_answers.append(x.lower())
    
N = len(correct_answers)
score = 0.0

for ans, cor in zip(found_answers, correct_answers):    
    if match(ans, cor):
        score += 1
        
print ('TOTAL SCORE:', score)

TOTAL SCORE: 3.0
